<a href="https://colab.research.google.com/github/jfmartineb/Intelifood/blob/main/InteliFood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Iteración Final

En este código se encuentra todo el modelo del agente trabjado durante el semestre

## Importe de librerias

In [ ]:
!pip install mesa
!pip install scikit-surprise
!pip install datasets
!pip install opencv-python
!pip install matplotlib
!pip install pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.1/107.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from mesa import Agent, Model
import csv
import random
import numpy as np
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans
from datasets import load_dataset
import ipywidgets as widgets
from IPython.display import display, Markdown
import cv2
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import tensorflow as tf
import requests
from IPython.display import Image
import pickle
from IPython.display import display, clear_output, HTML, display_html
from PIL import Image
import io

## Clase de Agente

In [ ]:
class Agente(Agent):
#___________________________Inicio del constructor______________________________
  def __init__(self):
    #Constantes necesarias para el algoritmo genético
    self.food_dict_by_food_code = {}
    self.food_dict_by_id = {}
    self.config = []
    self.INDEX_ID = 0
    self.INDEX_FOOD_CODE = 1
    self.INDEX_CALORIES = 5

    #Preparar los diccionarios necesarios para la recomendación del plan diario
    with open('food_listv2.csv', newline='', encoding='utf-8') as csvfile:
      reader = csv.reader(csvfile, delimiter=';', skipinitialspace=True)
      next(reader)
      for row in reader:
        food_code = row[self.INDEX_FOOD_CODE]
        id = row[self.INDEX_ID]
        if food_code in self.food_dict_by_food_code:
            self.food_dict_by_food_code[food_code].append(row)
        else:
            self.food_dict_by_food_code[food_code] = [row]
        self.food_dict_by_id[id] = row

    with open('configv2.csv', newline='', encoding='utf-8') as csvfile:
      reader = csv.reader(csvfile, delimiter=',')
      next(reader)
      for row in reader:
        self.config.append(row)

    #Crónicas definidas que evaluará el agente
    self.S = {
      # Se cumplió con las comidas mínimas
      "CM": {
        'e1': 'Levantarse',
        'e2': 'Comer',
        'e3': 'Comer'
      },
      # Se realizó ejercicio saludable.
      "ES": {
        'e1': 'Levantarse',
        'e2': 'Comer',
        'e3': 'Ejercicio'
      },
      # Se tuvo un día saludable
      "DS": {
        'e1': 'Levantarse',
        'e2': 'Comer',
        'e3': 'Ejercicio',
        'e4': 'Comer',
        'e5': 'Comer',
        'e6': 'Dormir',
        'e7': 'Comer',
        'e8': 'Ejercicio',
        'e9': 'Comer'
      }
    }

    #Lista de crónicas
    self.T = {
      "CM": {
        'e1': [('e2',0,24)],
        'e2': [('e3',1,12)]
      },
      "ES": {
        'e1': [('e2',0,24)],
        'e2': [('e3',4,24)]
      },
      "DS": {
        'e1': [('e2',1,2)],
        'e2': [('e3',1,5), ('e7',4,24)],
        'e3': [('e4',4,24)],
        'e4': [('e5',4,24)],
        'e5': [('e6',1,2)],
        'e7': [('e8',4,6)],
        'e8': [('e9',1.5,24)],
        'e9': [('e6',1,2)]
      }
    }

    # Prepara constantes necesarios y datasets para el algoritmo de aprendizaje
    # Especifica el nombre del conjunto de datos que deseas descargar
    DATASET_NAME = "Frorozcol/recetas-cocina"

    # Carga el conjunto de datos
    dataset = load_dataset(DATASET_NAME)

    self.df_recipes = pd.DataFrame(dataset['train'])
    self.nFilas, nColumnas = self.df_recipes.shape

    usuarios_origen = pd.read_csv('usernames.csv', sep = ',')
    self.platos = []
    self.usuarios = []
    self.calificaciones = []
    numeros = [1, 2, 3, 4, 5]
    pesos = [1, 2, 4, 4, 2]

    #Se llenan las listas necesarias para simular ratings de otros usuarios
    NUM_USUARIOS = 2500
    for index, row in usuarios_origen.iterrows():
      if index >= NUM_USUARIOS:
        break
      usuario = row['author']
      j = int(random.randint(5,10))
      for i in range(0,j):
        self.usuarios.append(usuario)
        self.platos.append(random.randint(0, self.nFilas))
        self.calificaciones.append(random.choices(numeros, pesos)[0])

    # Data inicial
    ratings_dict = {
        "dish": self.platos,
        "user": self.usuarios,
        "rating": self.calificaciones,
    }

    # Dataframe utilizado como dataset
    self.df = pd.DataFrame(ratings_dict)
    self.dishes = self.df['dish'].unique().tolist()
    self.users = self.df['user'].unique().tolist()

    # Número de registros de dataset simulado de redes bayesianas
    self.N_DATA = 100000

    # Se carga el modelo necesario para la predicción en el reconocimiento de imagenes
    self.model = pickle.load(open("model.pickle", "rb"))
    self.labels = pickle.load(open("labels.pickle", "rb"))

    self.app_id='de6ca170'
    self.app_key='a5dbda876e75f3d304b2bd0da1b560af'
    self.url = 'https://api.edamam.com/search'

#_____________________________Fin constructor __________________________________

#_____________Inicio de los métodos para el algoritmo genético_________________
  def getIndividual(self):
    individual = []
    for config_row in self.config:
        items_by_food_code = self.food_dict_by_food_code[config_row[0]]
        pos = random.randint(0, len(items_by_food_code)-1)
        value = (int(items_by_food_code[pos][self.INDEX_CALORIES]),int(items_by_food_code[pos][self.INDEX_ID]))
        individual.append(value)
    return individual

  def getPopulation(self):
    population = []
    for i in range(100):
        population.append(self.getIndividual())
    return population

  def getFitness(self, population, target_calories):
    fitness = []
    for individual in population:
        sum = 0
        for i in individual:
            sum+=i[0]
        value = abs(sum-target_calories)
        if value == 0:
            fitness+=[1]
        else:
            fitness+=[abs(sum-target_calories)]
    fitness = np.array(fitness)
    total = fitness.sum()
    fitness = fitness/total
    fitness = 1 / fitness
    fitness = fitness / fitness.sum()
    return fitness

  def runReproduction(self, population, target_calories):
    fitness = self.getFitness(population, target_calories)
    size_population = len(population)
    ind_size = len(population[0])
    #childs
    offspring = []
    for _ in range(size_population//2):
        parents = np.random.choice(a=size_population,size=2,p=fitness)
        cross_point = np.random.randint(ind_size)
        offspring += [population[parents[0]][:cross_point] + population[parents[1]][cross_point:]]
        offspring += [population[parents[1]][:cross_point] + population[parents[0]][cross_point:]]
    return offspring

  def mutate(self, individuals, prob):
    for i in range(len(individuals)):
        mutate_individual=individuals[i]
        if np.random.random() < prob:
            for j in range(0,len(mutate_individual)):
                if np.random.random() < prob:
                    items_by_food_code = self.food_dict_by_food_code[self.config[j][0]]
                    pos = random.randint(0, len(items_by_food_code)-1)
                    mutation =  (int(items_by_food_code[pos][self.INDEX_CALORIES]), items_by_food_code[pos][self.INDEX_ID])
                    mutate_individual = mutate_individual[0:j] + [mutation] + mutate_individual[j+1:]
            individuals[i] = mutate_individual
  def runGenerations(self, generations, target_calories, mutate_prob):
    population = self.getPopulation()
    for _ in range(generations):
        population = self.runReproduction(population, target_calories)
        self.mutate(population,mutate_prob)
    return population

  def chooseIndividual(self, result, target_calories):
    index = -1
    min = 100000000
    for i in range(0, len(result)):
      sum = 0
      for ca, fcode in result[i]:
        sum+=ca
      diff = abs(sum-target_calories)
      if(diff < min):
        min = diff
        index = i
    return result[i]

  def formatIndividual(self, result_item):
    sum = 0
    format = {
        'items': [],
        'total_k': 0
    }
    for i in range(0, len(self.config)):
      item = self.food_dict_by_id[str(result_item[i][1])]
      sum+=result_item[i][0]
      format['items'].append({
          'food_id': item[0],
          'food_type_code': item[1],
          'food_time': self.config[i][1],
          'food_type': item[2],
          'food_name': item[3]
      })
    format['total_k'] = sum
    return format

  def printPlan(self, format):
    result = 'Menú del día <br>'
    for i in format['items']:
      result = result + f"Momento del día: {i['food_time']} <br> Tipo de comida: {i['food_type']} <br> Comida: {i['food_name']} <br>"
    result = result + f"Total calorías: {format['total_k']}"
    return result

  def showPlan(self, format):
    result = 'Menú del día \n'
    for i in format['items']:
      result = result + f"Momento del día: {i['food_time']} \n Tipo de comida: {i['food_type']} \n Comida: {i['food_name']} \n"
    result = result + f"Total calorías: {format['total_k']}\n"
    return result
#_______________Fin de los métodos para el algoritmo genético___________________

#_____________Inicio de los métodos para el algoritmo crónicas__________________
  def evaluar_grafo(self, grafo, ruta):
    flag = True
    last = 0
    for i in range(0, len(ruta) - 1):
      flag = False
      last = i
      for j in grafo[ruta[i]]:
        if j[0] == ruta[i+1]:
          flag = True
      if not flag:
        return False
    if ruta[last] in grafo:
      return False
    return True

  def encontrar_evento(self, evento, eventos, cumplidos):
    for i in eventos.keys():
      if eventos[i] == evento and i not in cumplidos:
        return i

  def evaluar(self, clave, entradas):
    i = 0
    ev_cumplidos = []
    ti = 0
    to = 0
    flag = True
    print(clave)
    print(entradas)
    cronica = self.T[clave]
    eventos = self.S[clave]
    if entradas[i][0] != 'Levantarse':
      return False
    ti = entradas[i][1]
    sig_evento = self.encontrar_evento(entradas[i][0], eventos, ev_cumplidos)
    while True:
      if flag:
        ev_cumplidos.append(sig_evento)
        try:
          sig = cronica[ev_cumplidos[-1]]
        except KeyError:
          break
      i += 1
      if i >= len(entradas):
        break
      to = entradas[i][1]
      for j in sig:
        if eventos[j[0]] == entradas[i][0]:
          t_tareas = to - ti
          if j[1] <= t_tareas and t_tareas <= j[2]:
            ti = to
            flag = True
            sig_evento = j[0]
            break
          else:
            flag = False
        else:
          flag = False

    print(ev_cumplidos)
    return self.evaluar_grafo(cronica, ev_cumplidos)
#_______________Fin de los métodos para el algoritmo crónicas___________________

#_______Inicio de los métodos para el algoritmo de redes bayesianas_____________

  # Función para generación de dataset
  def generate_dataset(self):
    # Establecemos cada una de las probabilides de éxito en cada una de las variables aleatorias definidas.
    p_aca = 0.8
    p_acgs = 0.8
    p_acap = 0.8
    p_to = 0.43
    p_tb = 0.23
    # Retornamos un diccionarío con cada una de las muestras de las variables aleatorias.
    return {
        "aca": np.random.choice([0, 1], size=self.N_DATA, p=[1-p_aca, p_aca]),
        "acgs": np.random.choice([0, 1], size=self.N_DATA, p=[1-p_acgs, p_acgs]),
        "acap": np.random.choice([0, 1], size=self.N_DATA, p=[1-p_acap, p_acap]),
        "to": np.random.choice([0, 1], size=self.N_DATA, p=[1-p_to, p_to]),
        "tb": np.random.choice([0, 1], size=self.N_DATA, p=[1-p_tb, p_tb])
    };

  # Función para encontrar la probabilidad a priori de las variables aleatorias sin padre.
  def probabilidad_priori(self, variable, dataset):
    return len(list(filter(lambda x: x == 1, dataset[variable])))/self.N_DATA

  # Función para encontrar la probabilidad condicional de las variables aleatorias con tres padres.
  def probabilidad_condicional_3(self, cond1, cond2, cond3, cond4, dataset):
    a = 0
    b = 0
    for i in range(self.N_DATA):
      if (
        dataset[cond1[0]][i] == cond1[1]
        and dataset[cond2[0]][i] == cond2[1]
        and dataset[cond3[0]][i] == cond3[1]
        and dataset[cond4[0]][i] == cond4[1]
      ):
        a = a + 1
      if (
        dataset[cond2[0]][i] == cond2[1]
        and dataset[cond3[0]][i] == cond3[1]
        and dataset[cond4[0]][i] == cond4[1]
      ):
        b = b + 1
    return a / b

  # Función para encontrar la probabilidad condicional de las variables aleatorias con un padre.
  def probabilidad_condicional_1(self, cond1, cond2, dataset):
    a = 0
    b = 0
    for i in range(self.N_DATA):
      if (
        dataset[cond1[0]][i] == cond1[1]
        and dataset[cond2[0]][i] == cond2[1]
      ):
        a = a + 1
      if (
        dataset[cond2[0]][i] == cond2[1]
      ):
        b = b + 1
    return a / b

  # Función que encuentra la probabilidad conjunta de todas las variables aleatorias
  def probabilidad_conjunta(self, v1,v2,v3,v4,v5,dataset):
    probabilidades = {
      "p(aca)": self.probabilidad_priori("aca",dataset),
      "p(acgs)": self.probabilidad_priori("acgs",dataset),
      "p(acap)": self.probabilidad_priori("acap",dataset),
      "p(to|aca,acgs,acap)": self.probabilidad_condicional_3(("to",1),("aca",1),("acgs",1),("acap",1), dataset),
      "p(to|aca,acgs,-acap)": self.probabilidad_condicional_3(("to",1),("aca",1),("acgs",1),("acap",0), dataset),
      "p(to|aca,-acgs,acap)": self.probabilidad_condicional_3(("to",1),("aca",1),("acgs",0),("acap",1), dataset),
      "p(to|aca,-acgs,-acap)": self.probabilidad_condicional_3(("to",1),("aca",1),("acgs",0),("acap",0), dataset),
      "p(to|-aca,acgs,acap)": self.probabilidad_condicional_3(("to",1),("aca",0),("acgs",1),("acap",1), dataset),
      "p(to|-aca,acgs,-acap)": self.probabilidad_condicional_3(("to",1),("aca",0),("acgs",1),("acap",0), dataset),
      "p(to|-aca,-acgs,acap)": self.probabilidad_condicional_3(("to",1),("aca",0),("acgs",0),("acap",1), dataset),
      "p(to|-aca,-acgs,-acap)": self.probabilidad_condicional_3(("to",1),("aca",0),("acgs",0),("acap",0), dataset),
      "p(tb|aca)": self.probabilidad_condicional_1(("tb",1),("aca",1),dataset),
      "p(tb|-aca)": self.probabilidad_condicional_1(("tb",1),("aca",0),dataset)
    }

    per = 1
    if v1:
      per = per * probabilidades["p(aca)"]
    else:
      per = per * (1 - probabilidades["p(aca)"])
    if v2:
      per = per * probabilidades["p(acgs)"]
    else:
      per = per * (1 - probabilidades["p(acgs)"])

    if v3:
      per = per * probabilidades["p(acap)"]
    else:
      per = per * (1 - probabilidades["p(acap)"])

    aca = "aca" if v1 else "-aca"
    acgs = "acgs" if v2 else "-acgs"
    acap = "acap" if v3 else "-acap"
    key_v4 = "p(to|"+aca+","+acgs+","+acap+")"
    key_v5 = "p(tb|"+aca+")"

    if v4:
      per = per * probabilidades[key_v4]
    else:
      per = per * (1 - probabilidades[key_v4])

    if v5:
      per = per * probabilidades[key_v5]
    else:
      per = per * (1 - probabilidades[key_v5])
    return per
#________Fin de los métodos para el algoritmo de redes bayesianas_______________

#__________Inicio de los métodos para el algoritmo de aprendizaje_______________
  def calculate_rating(self, user, dishes_):
    # Establecemos el rango de los ratings
    ratings = []
    reader = Reader(rating_scale=(1, 5))
    # Cargamos el conjuntos de datos y el reader para interpretar los ratings
    data = Dataset.load_from_df(self.df[["user", "dish", "rating"]], reader)
    # Establecemos las opciones de configuración del algortimo a utilizar
    # En este caso establecemos que vamos a utilizar como método para encontrar similitud "cosine"
    # y establecemos a la propiedad user_based en False porque queremos que sea por items y no por usuarios
    sim_options = {
        "name": "cosine",
        "user_based": False,
    }
    # Carga de configuración y entrenamiento con datos
    algo = KNNWithMeans(sim_options=sim_options)
    trainingSet = data.build_full_trainset()
    algo.fit(trainingSet)

    #Realizamos predicción y la retornamos
    for i in dishes_:
      prediction = algo.predict(user, i)
      ratings.append(prediction.est)
    return ratings

  def recommend_dish(self, max_dishes, user):
    new_dishes = []
    for i in self.dishes:
      history = self.df[(self.df['user'] == user) & (self.df['dish'] == i)]
      if history.empty:
        new_dishes.append(i)

    ratings = self.calculate_rating(user, new_dishes)
    dicc = {
        'dish': new_dishes,
        'rating': ratings
    }
    recommend = pd.DataFrame(dicc)
    recommend = recommend.sort_values(by='rating', ascending=False)
    print(recommend['dish'].iloc[:max_dishes].tolist())
    return recommend['dish'].iloc[:max_dishes].tolist()

  def set_rating(self, user, dish, rating):
    try:
      # Find actual ratings for user
      df_user = self.df[ self.df['user'] == user ]
      # Find if the user already rate the dish and get the index
      df_dish = df_user[ df_user['dish'] == dish ].index.values
      # Replace the rating value if exists otherwise add new rating
      i = df_dish[0] if len(df_dish) else df.index[-1] + 1
      self.df.loc[i] = pd.Series({'dish': dish, 'user': user, 'rating': rating})
      if dish not in self.dishes:
        self.dishes.append(dish)
      if user not in self.users:
        self.users.append(user)
      return True
    except:
      return False

  def info_receta(self, num):
    return self.df_recipes.iloc[num].tolist()

  def user_exists(self, user):
    if user in self.users:
      return True
    return False
#____________Fin de los métodos para el algoritmo de aprendizaje________________

#________Inicio de los métodos para el algoritmo de reconocimiento______________
  def segmentar_imagen(self, file):
    #Se abre la imagen
    #img = cv2.imread(path)
    pil_image = Image.open(io.BytesIO(file[next(iter(file))]['content']))
    cv2_image = np.array(pil_image)
    img = cv2.cvtColor(cv2_image, cv2.COLOR_RGB2BGR)
    imagenes = []

    # Con la libreria de OpenCV se descarga ña imagen en formato BGR y se convierte a RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width = img.shape[:2]
    area = height * width
    #Se formatea la imagen a un arreglo 2D depixeles y 3 colores (RGB)
    pixel_vals = img_rgb.reshape((-1,3)) # numpy reshape operation -1 unspecified
    # Se convierte a un valor de float para que sea compatible con la funcion de kmeans
    pixel_vals = np.float32(pixel_vals)

    # Criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.85)

    #Se escoge el número de cluster que se formarán inicialmente
    k = 5
    #Se aplica la función de kmeans
    retval, labels, centers = cv2.kmeans(pixel_vals, k, None, criteria, 5, cv2.KMEANS_RANDOM_CENTERS)
    #Se convierte la data a pixeles enteros
    centers = np.uint8(centers)

    #Se mapean las marcas con los centros
    segmented_data = centers[labels.flatten()]

    # Se devuelve la información al mismo tamaño orignal
    segmented_image = segmented_data.reshape((img_rgb.shape))

    image = segmented_image

    #Se aplican funciones que ayudan a dilatar la imagen primero a escala de grises y luego
    #a blanco y negro, para facilitar encontrar los contornos de la imagen
    gKernel = 5
    sigmaX = 0

    original = img.copy()
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (gKernel, gKernel), sigmaX)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    dilate = cv2.dilate(thresh, kernel, iterations=1)

    #Se encuentran los contornos, que luego serán enmarcados con el objetivo de sacar las diferentes
    #objetos que se encuentren para su debida clasificación
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    image_number = 0
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        #Se filtran todos los rectangulos y se verifica que ocupen más del 1% de la imagen
        sobx = int((w * 0.1)/2)
        soby = int((h *0.1)/2)
        if (w*h)/area > 0.01:
          x0 = x - sobx
          if x0 < 0:
            x0 = 0 + 5
          y0 =  y - soby
          if y0 < 0:
            y0 = 0 + 5
          x1 = (x + w) + sobx
          if x1 > width:
            x1 = width -5
          y1 = (y + h) + soby
          if y1 > height:
            y1 = height -5
          cv2.rectangle(img, (x0, y0), (x1, y1), (36,255,12), 2)
          ROI = original[y0:y1, x0:x1]
          imagenes.append(ROI)
          image_number += 1

    cv2_imshow(img)

    #Se devuelven todas las imagenes como un arreglo de imagenes de cv2
    return imagenes

  def preprocess(self, image):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    image = image[None, ...]
    return image

  def run_detectorV3(self, path):
    #img = load_img(path)
    images = self.segmentar_imagen(path)
    results = []

    for i in images:
      img = self.preprocess(i)
      ge_probs = self.model.predict(img)
      pred_index = np.argmax(ge_probs,axis=1)
      pred = self.labels[pred_index].tolist()[0]
      results.append(pred)

    return results

  def get_recipes(self, ingredients, fr, to):
    params = {'app_id': self.app_id, 'app_key': self.app_key, 'q': ingredients, 'from': fr, 'to': to }
    response = requests.get(self.url, params=params)
    if response.status_code == 200:
      data = response.json()
      return data['hits']
    else:
      print(f'Error {response.status_code}: {response.text}')

  def show_recipes(self, recipes):
    for recipe in recipes:
      print('*'*50)
      print(recipe['recipe']['label'])
      print('*'*50)
      print('Ingredients\n')
      for i in recipe['recipe']['ingredientLines']:
        print('- ',i)
      print('*'*50)

#_________Fin de los métodos para el algoritmo de reconocimiento________________

NameError: ignored






## Modelo

In [ ]:
class Modelo(Model):
  def __init__(self, Agente, usuario_,):
    self.intelifood = Agente
    self.user = usuario_

  # Métodos necesarios para utilizar el algoritmo genético
  def recommendDay(self, generations, target_calories, mutate_prob):
    result = self.intelifood.runGenerations(generations, target_calories, mutate_prob)
    individual = self.intelifood.chooseIndividual(result,target_calories)
    result_format = self.intelifood.formatIndividual(individual)
    return self.intelifood.printPlan(result_format)

  #Métodos para evaluar una crónica
  def evaluateEvent(self, chronic, events):
    self.intelifood.evaluar(chronic, events)

  #Método para probabilidad de enfermedad
  def getProbabilityOfDisease(self, has_aca, has_acgs, has_acap, has_to, hast_tb):
    dataset = self.intelifood.generate_dataset()
    return self.intelifood.probabilidad_conjunta(has_aca,has_acgs,has_acap,has_to,hast_tb,dataset)

  #Métodos para recomendar platos con el aprendizaje
  def recommendDish(self, num):
    if not self.intelifood.user_exists(self.user):
      self.addRecipes(5)
    dishes = self.intelifood.recommend_dish(num, self.user)
    output = f"{dishes[0]} - {self.intelifood.info_receta(dishes[0])[0]}"
    for dish in dishes[1:]:
      output = f"{output}, {dish} - {self.intelifood.info_receta(dish)[0]}"
    return output

  def addRecipes(self, num):
    for i in range(0, num):
      dish = random.randint(0, self.intelifood.nFilas)
      rating = int(input(f"Da un rating para la receta {self.intelifood.info_receta(dish)[0]}: "))
      self.setRating(dish, rating)

  def setRating(self, recipe, rating):
    return self.intelifood.set_rating(self.user, recipe, rating)

  #Métodos para la recomendación de platos desde una imágen
  def recommendDishByImage(self, name):
    ingredients = self.intelifood.run_detectorV3(name)
    recipes = self.intelifood.get_recipes(ingredients, 0, 5)
    self.intelifood.show_recipes(recipes)


NameError: ignored

## Declaración del agente


In [ ]:
intelifood = Agente()

NameError: ignored

# Interface

In [ ]:

user = None

# actions
def configUser(button):
  global user
  user = Modelo(intelifood, text_config.value)
  response_config.value = 'Usuario configurado correctamente'

def mealPlan(button):
  calories = text_meal_plan.value
  result = user.recommendDay(100, int(calories), 0.1)
  response_meal_plan.value = result

def add_dropdowns(button):
  new_dropdowns = (widgets.Dropdown(
                      options=['Levantarse', 'Comer', 'Ejercicio', 'Trabajar', 'Estudiar', 'Ocio'],
                      description='Actividad: '),
                   widgets.BoundedIntText(placeholder = "Hora del día", min=0, max=24, description='Hora del día: '))
  container_events.children += new_dropdowns

def recommendByImage(button):
  file = file_upload_recomment_by_image.value
  user.recommendDishByImage(file)

def probabilityDiseases(button):
  prob = user.getProbabilityOfDisease(
      check_box_diseases_aca.value,
      check_box_diseases_acgs.value,
      check_box_diseases_acap.value,
      check_box_diseases_to.value,
      check_box_diseases_tb.value
  )
  response_diseases.value = f'La probabilidad de que se cumpla {prob*100}%'

def recommendByUser(button):
  response_1_recommend_by_users.value = 'Cargando...'
  result = user.recommendDish(text_1_recommend_by_users.value)
  response_1_recommend_by_users.value = str(result)

def delete_dropdowns(button):
  container_events.children = container_events.children[0:4]

def evaluate_life_style(button):
  selected_values = [dropdown.value for dropdown in container_events.children[4:]]
  chronic = []
  for i in range(0, len(selected_values), 2):
    tuple_c = (selected_values[i], selected_values[i+1])
    chronic.append(tuple_c)
  response = user.evaluateEvent(dropdown_ls_intro.value, chronic)
  container_events.children = container_events.children[0:4]
  if not response:
    mns = (widgets.Label(value='Cumpliste con tu propoósito'), widgets.Label(value=' '))
  else:
    mns = (widgets.Label(value='No cumpliste con tu propoósito'), widgets.Label(value=' '))
  container_events.children += mns

# titles
display(Markdown("# Bienvenido a InteliFood"))
display(Markdown("## En la parte inferior tendrás cada una de nuestras opciones"))

# content
# Config
title_config = widgets.Label(value='Ingresa tú nombre de usuario')
text_config = widgets.Text(placeholder = "Nombre de usuario")
button_config = widgets.Button( description='Guardar',disabled=False,button_style='success',tooltip='Guardar usuario')
button_config.on_click(configUser)
response_config = widgets.HTML(value='Response: ')

# meal plan content
title_meal_plan = widgets.Label(value='Ingresa cuantas de cuantas calorías quieres tú plan')
text_meal_plan = widgets.BoundedIntText(placeholder = "Calorías", min=200, max=5000)
button_meal_plan = widgets.Button( description='Generar',disabled=False,button_style='success',tooltip='Generar plan')
button_meal_plan.on_click(mealPlan)
response_meal_plan = widgets.HTML(value='Response: ')

# life style content
title_ls_intro = widgets.Label(value='¿Que quieres eavaluar de tu día?')
dropdown_ls_intro = widgets.Dropdown(
                  options=[('¿Cumpliste con las comidas mínimas?', 'CM'),
                           ('¿Hiciste ejercicio cuando se debía?', 'ES'),
                           ('Tuviste un día saludable?', 'DS')],
                  description='Number:')
title_ls_events = widgets.Label(value='Añade eventos: ')
button_ls_event = widgets.Button(description='Añade evento')
button_ls_event.on_click(add_dropdowns)
button_ls_evaluate = widgets.Button(description='Evaluar')
button_ls_evaluate.on_click(evaluate_life_style)
button_ls_eliminate = widgets.Button(description='Eliminar')
button_ls_eliminate.on_click(delete_dropdowns)


# diseases content
title_diseases = widgets.Label(value='Responde las preguntas y miremos que probabilidad tienes de tener obesidad y diabetes')
title_diseases_aca = widgets.Label(value='¿Consideras que tienes alto consumo de azucar?')
check_box_diseases_aca = widgets.Checkbox(
    value=False,
    disabled=False
)
title_diseases_acgs = widgets.Label(value='¿Consideras que tienes alto consumo de grasas saturadas?')
check_box_diseases_acgs = widgets.Checkbox(
    value=False,
    disabled=False
)
title_diseases_acap = widgets.Label(value='¿Consideras que tienes alto consumo de alimentos procesados?')
check_box_diseases_acap = widgets.Checkbox(
    value=False,
    disabled=False
)
title_diseases_options = widgets.Label(value='¿Que enfermedad deseas saber? Puedes seleccionar las dos si deseas')
check_box_diseases_to = widgets.Checkbox(
    value=False,
    description='Obesidad',
    disabled=False
)
check_box_diseases_tb = widgets.Checkbox(
    value=False,
    description='Diabetes',
    disabled=False
)
button_diseases = widgets.Button( description='Calcular',disabled=False,button_style='success',tooltip='Calcular probabilidad')
button_diseases.on_click(probabilityDiseases)
response_diseases = widgets.Label(value='Response: ')

# recommend by users content
title_1_recommend_by_users = widgets.Label(value='Recomendar')
title_2_recommend_by_users = widgets.Label(value='Ingresa el número de platos que deseas')
text_1_recommend_by_users = widgets.BoundedIntText(placeholder = "Platos", min=1, max=10)
button_1_recommend_by_users = widgets.Button( description='Generar',disabled=False,button_style='success',tooltip='Generar platos')
button_1_recommend_by_users.on_click(recommendByUser)
response_1_recommend_by_users = widgets.Label(value='Response: ')
title2_recommend_by_users = widgets.Label(value='Establecer rating')

# recommend by image content
title_recomment_by_image = widgets.Label(value='Sube una imagen con los ingredientes que cuentas para darte algunas sugerencias de platos')
file_upload_recomment_by_image = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
button_recomment_by_image = widgets.Button( description='Generar',disabled=False,button_style='success',tooltip='Generar plan')
button_recomment_by_image.on_click(recommendByImage)

#containers
container_config = widgets.VBox([title_config, text_config, button_config, response_config])
container_meal_plan = widgets.VBox([title_meal_plan, text_meal_plan, button_meal_plan, response_meal_plan])
container_events = widgets.VBox([title_ls_events, button_ls_event,  button_ls_eliminate, button_ls_evaluate])
container_life_style = widgets.VBox([title_ls_intro, dropdown_ls_intro, container_events])
container_diseases = widgets.VBox([
    title_diseases,
    title_diseases_aca,
    check_box_diseases_aca,
    title_diseases_acgs,
    check_box_diseases_acgs,
    title_diseases_acap,
    check_box_diseases_acap,
    title_diseases_options,
    check_box_diseases_to,
    check_box_diseases_tb,
    button_diseases,
    response_diseases
])
container_recommend_by_users = widgets.VBox([
    title_1_recommend_by_users,
    title_2_recommend_by_users,
    text_1_recommend_by_users,
    button_1_recommend_by_users,
    response_1_recommend_by_users,
    title2_recommend_by_users
    ])
container_recommend_by_image = widgets.VBox([title_recomment_by_image, file_upload_recomment_by_image, button_recomment_by_image])

# Create tabs
tabs = widgets.Tab(children=[container_config, container_meal_plan, container_life_style, container_diseases, container_recommend_by_users, container_recommend_by_image])

# Set labels to tabs
tabs.set_title(0, 'Configuración')
tabs.set_title(1, 'Plan alimenticio')
tabs.set_title(2, 'Estilo de vida')
tabs.set_title(3, 'Enfermedades')
tabs.set_title(4, 'Recom. por usuario')
tabs.set_title(5, 'Recom. por imagen')

# Show tabs
display(tabs)

# Bienvenido a InteliFood

## En la parte inferior tendrás cada una de nuestras opciones